<a href="https://colab.research.google.com/github/rishabhshah13/DCP_Capstone/blob/main/des_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

import tensorflow as tf
import pandas as pd
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from torch import cuda
from torch.utils.data import Dataset
from transformers import TrainingArguments, Trainer
import torch
from google.colab import drive


device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
drive.mount('/content/drive')


# df = pd.read_csv('/Users/shuai/Desktop/combined_df')
df = pd.read_csv('/content/final_df_for_training.csv')

df['relevant'] = df['relevant'].replace({'no': '0', 'o': '0'}, regex=True)

df.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(1572, 2)

In [ ]:
labels = df['relevant'].unique().tolist()

NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}
df["labels"]=df.relevant.map(lambda x: label2id[x.strip()])
df.head()

,relevant,Company Li Description,labels
0,0,Jam is where audio comes alive ‚Äì the new way...,0
1,0,Those who can - the creatives network. The on...,0
2,0,Our Mission: Enable innovation for high perfo...,0
3,0,"Especializate en Influencer Marketing, con foc...",0
4,1,"Oasis X, is a spring of creativity in the web3...",1


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("dbmdz/bert-base-turkish-uncased", max_length=512)
model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-uncased", num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
df = df.dropna()
data_texts = df['Company Li Description'].to_list()

data_labels = df['labels'].to_list()

train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.2, random_state = 0 )
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.01, random_state = 0 )

len(train_texts), len(val_texts), len(test_texts)

(1244, 315, 13)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings  = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
class DataLoader(Dataset):
    """
    Custom Dataset class for handling tokenized text data and corresponding labels.
    Inherits from torch.utils.data.Dataset.
    """
    def __init__(self, encodings, labels):
        """
        Initializes the DataLoader class with encodings and labels.

        Args:
            encodings (dict): A dictionary containing tokenized input text data
                              (e.g., 'input_ids', 'token_type_ids', 'attention_mask').
            labels (list): A list of integer labels for the input text data.
        """
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        """
        Returns a dictionary containing tokenized data and the corresponding label for a given index.

        Args:
            idx (int): The index of the data item to retrieve.

        Returns:
            item (dict): A dictionary containing the tokenized data and the corresponding label.
        """
        # Retrieve tokenized data for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the label for the given index to the item dictionary
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        """
        Returns the number of data items in the dataset.

        Returns:
            (int): The number of data items in the dataset.
        """
        return len(self.labels)

train_dataloader = DataLoader(train_encodings, train_labels)

val_dataloader = DataLoader(val_encodings, val_labels)

test_dataset = DataLoader(test_encodings, test_labels)

In [ ]:
def compute_metrics(pred):
    """
    Computes accuracy, F1, precision, and recall for a given set of predictions.

    Args:
        pred (obj): An object containing label_ids and predictions attributes.
            - label_ids (array-like): A 1D array of true class labels.
            - predictions (array-like): A 2D array where each row represents
              an observation, and each column represents the probability of
              that observation belonging to a certain class.

    Returns:
        dict: A dictionary containing the following metrics:
            - Accuracy (float): The proportion of correctly classified instances.
            - F1 (float): The macro F1 score, which is the harmonic mean of precision
              and recall. Macro averaging calculates the metric independently for
              each class and then takes the average.
            - Precision (float): The macro precision, which is the number of true
              positives divided by the sum of true positives and false positives.
            - Recall (float): The macro recall, which is the number of true positives
              divided by the sum of true positives and false negatives.
    """
    # Extract true labels from the input object
    labels = pred.label_ids

    # Obtain predicted class labels by finding the column index with the maximum probability
    preds = pred.predictions.argmax(-1)

    # Compute macro precision, recall, and F1 score using sklearn's precision_recall_fscore_support function
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')

    acc = accuracy_score(labels, preds)

    # Return the computed metrics as a dictionary
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [ ]:

training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='./TTC4900Model',
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
   # TensorBoard log directory
    logging_dir='./multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    fp16=False,
    load_best_model_at_end=True
)

trainer = Trainer(
    # the pre-trained model that will be fine-tuned
    model=model,
     # training arguments that we defined above
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=val_dataloader,
    compute_metrics= compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()



Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.859400,0.786597,0.688889,0.271930,0.229630,0.333333
100,0.717300,0.758631,0.653968,0.434421,0.409561,0.512289
150,0.673600,0.629796,0.765079,0.502701,0.467078,0.546851
200,0.589400,0.649420,0.780952,0.514161,0.482339,0.550691
250,0.486800,0.653447,0.761905,0.504402,0.469915,0.549155
300,0.391600,0.802648,0.765079,0.505769,0.467296,0.562212
350,0.379800,0.873390,0.736508,0.523058,0.520082,0.567674
400,0.248800,0.832954,0.765079,0.570906,0.612961,0.550947
450,0.189500,0.896849,0.780952,0.559345,0.612602,0.558500
500,0.124700,0.932762,0.768254,0.604667,0.616740,0.594854


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

TrainOutput(global_step=624, training_loss=0.3901139547427495, metrics={'train_runtime': 334.209, 'train_samples_per_second': 29.778, 'train_steps_per_second': 1.867, 'total_flos': 2618504733229056.0, 'train_loss': 0.3901139547427495, 'epoch': 8.0})

In [ ]:
def predict(text):
    """
    Predicts the class label for a given input text

    Args:
        text (str): The input text for which the class label needs to be predicted.

    Returns:
        pred_label (str): The predicted class label.
    """
    # Tokenize the input text and move tensors to the GPU if available
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

    # Move the tensors to the same device as the model
    inputs = inputs.to("cuda")

    # Ensure the model is in evaluation mode and on the correct device
    model.eval()
    model.to("cuda")

    # Get model output (logits)
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    """ Explanation outputs: The BERT model returns a tuple containing the output logits (and possibly other elements depending on the model configuration). In this case, the output logits are the first element in the tuple, which is why we access it using outputs[0].

    outputs[0]: This is a tensor containing the raw output logits for each class. The shape of the tensor is (batch_size, num_classes) where batch_size is the number of input samples (in this case, 1, as we are predicting for a single input text) and num_classes is the number of target classes.

    softmax(1): The softmax function is applied along dimension 1 (the class dimension) to convert the raw logits into class probabilities. Softmax normalizes the logits so that they sum to 1, making them interpretable as probabilities. """

    # Get the index of the class with the highest probability
    # argmax() finds the index of the maximum value in the tensor along a specified dimension.
    # By default, if no dimension is specified, it returns the index of the maximum value in the flattened tensor.
    pred_label_idx = probs.argmax()

    # Now map the predicted class index to the actual class label
    # Since pred_label_idx is a tensor containing a single value (the predicted class index),
    # the .item() method is used to extract the value as a scalar
    pred_label = model.config.id2label[pred_label_idx.item()]

    return pred_label


In [ ]:
text = "Revolutionizing Material Science with Cutting-Edge Machine Learning Solutions. "
predict(text)

'1'

In [ ]:
model_path = "des-classification-model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

KeyboardInterrupt: 

In [ ]:
whole_df = pd.read_csv('/content/whole_df.csv')


In [ ]:
whole_df.head()

,Unnamed: 0,Email Status,Email,Lead First Name,Lead Last Name,Lead Full Name,Lead Gender,Lead Job Title,Lead Linkedin Url,Company Name,...,Company Li Description,Company Followers,Lead Summary,Lead Years In Position,Lead Months In Position,Lead Years In Company,Lead Months In Company,Lead Position Started Month,Lead Position Started Year,Company Location Country Name
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,Jason,Chatterjee,Jason Chatterjee,male,NaN,https://www.linkedin.com/in/ACwAAAC4EdEBAiqC6E...,NaN,...,NaN,NaN,As a CFO and experienced Private Equity invest...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,Brent,Kao,Brent Kao,male,Managing Director,https://www.linkedin.com/in/ACwAAAOKupgBy4LVAR...,Metlife Investment Management,...,MetLife Investment Management serves instituti...,11490.0,18+ years experience in financial service indu...,1.0,6.0,1.0,6.0,10.0,2022.0,United States
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import numpy as np
def clean_data(df1):

    df1 = df1.drop(columns=['Lead Gender'])

    # Replace missing values in the columns with 'Null' or 'Unknown'
    df1['Email Status'] = df1['Email Status'].fillna('Unknown')
    df1['Email'] = df1['Email'].fillna('Unknown')
    df1['Lead Job Title'] = df1['Lead Job Title'].fillna('Unknown')
    df1['Company Name'] = df1['Company Name'].fillna('Unknown')
    df1['Company Website'] = df1['Company Website'].fillna('Unknown')
    df1['Company Industry'] = df1['Company Industry'].fillna('Unknown')
    df1['Lead Linkedin Url'] = df1['Lead Linkedin Url'].fillna('Unknown')
    df1['Company Linkedin'] = df1['Company Linkedin'].fillna('Unknown')
    df1['Company Li Description'] = df1['Company Li Description'].fillna('Unknown')
    df1['Company Specialities'] = df1['Company Specialities'].fillna('Unknown')
    df1['Lead First Name'] = df1['Lead First Name'].fillna('Unknown')
    df1['Lead Last Name'] = df1['Lead Last Name'].fillna('Unknown')
    df1['Lead Full Name'] = df1['Lead Full Name'].fillna('Unknown')




    df1['Lead Summary'] = df1['Lead Summary'].fillna('Unknown')
    df1['Company Location Country Name'] = df1['Company Location Country Name'].fillna('Unknown')


    # Replace missing values in the columns with most frequently occurring value
    df1['Lead Years In Position'] = df1['Lead Years In Position'].fillna(df1['Lead Years In Position'].mode()[0])
    df1['Lead Months In Position'] = df1['Lead Months In Position'].fillna(df1['Lead Months In Position'].mode()[0])
    df1['Lead Years In Company'] = df1['Lead Years In Company'].fillna(df1['Lead Years In Company'].mode()[0])
    df1['Lead Months In Company'] = df1['Lead Months In Company'].fillna(df1['Lead Months In Company'].mode()[0])
    df1['Lead Position Started Month'] = df1['Lead Position Started Month'].fillna(df1['Lead Position Started Month'].mode()[0])
    df1['Lead Position Started Year'] = df1['Lead Position Started Year'].fillna(df1['Lead Position Started Year'].mode()[0])
    df1['Company Size'] = df1['Company Size'].fillna(df1['Company Size'].mode()[0])
    df1['Company Founded In'] = df1['Company Founded In'].fillna(df1['Company Founded In'].mode()[0])
    df1['Company Li Company Type'] = df1['Company Li Company Type'].fillna(df1['Company Li Company Type'].mode()[0])


    # Replace missing values in the column with the mean
    df1['Company Followers'] = np.ceil(df1['Company Followers'].fillna(df1['Company Followers'].mean())).astype(int)

    return df1

    # # Construct the path for the cleaned file
    # file_directory, file_name = os.path.split(csv_file)
    # cleaned_file_name = "cleaned_" + file_name
    # cleaned_csv_file = os.path.join(file_directory, cleaned_file_name)

    # # Save the cleaned DataFrame
    # df1.to_csv(cleaned_csv_file, index=False)

In [ ]:
cleaned_whole_df = clean_data(whole_df)

In [ ]:
cleaned_whole_df.head()

,Unnamed: 0,Email Status,Email,Lead First Name,Lead Last Name,Lead Full Name,Lead Job Title,Lead Linkedin Url,Company Name,Company Website,...,Company Li Description,Company Followers,Lead Summary,Lead Years In Position,Lead Months In Position,Lead Years In Company,Lead Months In Company,Lead Position Started Month,Lead Position Started Year,Company Location Country Name
0,0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,198000,Unknown,1.0,3.0,1.0,3.0,1.0,2023.0,Unknown
1,1,Unknown,Unknown,Jason,Chatterjee,Jason Chatterjee,Unknown,https://www.linkedin.com/in/ACwAAAC4EdEBAiqC6E...,Unknown,Unknown,...,Unknown,198000,As a CFO and experienced Private Equity invest...,1.0,3.0,1.0,3.0,1.0,2023.0,Unknown
2,2,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,198000,Unknown,1.0,3.0,1.0,3.0,1.0,2023.0,Unknown
3,3,Unknown,Unknown,Brent,Kao,Brent Kao,Managing Director,https://www.linkedin.com/in/ACwAAAOKupgBy4LVAR...,Metlife Investment Management,https://investments.metlife.com,...,MetLife Investment Management serves instituti...,11490,18+ years experience in financial service indu...,1.0,6.0,1.0,6.0,10.0,2022.0,United States
4,4,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,...,Unknown,198000,Unknown,1.0,3.0,1.0,3.0,1.0,2023.0,Unknown


In [ ]:
nan_counts = cleaned_whole_df.isna().sum()
print(nan_counts)


Unnamed: 0                       0
Email Status                     0
Email                            0
Lead First Name                  0
Lead Last Name                   0
Lead Full Name                   0
Lead Job Title                   0
Lead Linkedin Url                0
Company Name                     0
Company Website                  0
Company Size                     0
Company Industry                 0
Company Linkedin                 0
Company Founded In               0
Company Specialities             0
Company Li Company Type          0
Company Li Description           0
Company Followers                0
Lead Summary                     0
Lead Years In Position           0
Lead Months In Position          0
Lead Years In Company            0
Lead Months In Company           0
Lead Position Started Month      0
Lead Position Started Year       0
Company Location Country Name    0
dtype: int64


In [ ]:
cleaned_whole_df['Company Des Relevant Score'] = cleaned_whole_df['Company Li Description'].apply(predict)


In [ ]:
cleaned_whole_df.to_csv('/content/cleaned_whole_df.csv', index=False)